In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.metrics import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM
from tensorflow.keras.optimizers import SGD
from keras.utils.vis_utils import plot_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

import numpy.random as rand
import scipy.stats as stats 
plt.style.use('ggplot')
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor 
from sklearn.model_selection import StratifiedKFold

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta, time, datetime
import time

Use root mean squared log error

Pipeline?

Stratified KFold Split (train test split)

In [ ]:
skf = StratifiedKFold()
skf.get_n_splits(X, y)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

X_train = combine all weeks into one dataframe, decompose into closing(trend_1 + trend_2), volume(trend), +1 column, add additional day as prediction happens to predict next day

y_train = closing(trend_1 + trend_2)? target is next minute price? min, max of next day?

neural network model

In [ ]:
model = Sequential()
model.add(LSTM(units=10, return_sequences=True, input_shape=(???)))
model.add(Dropout())

In [ ]:
def buildNN(X, y, activation_function, metric, optimizer, hidden_layers=[7], batch_size=32, epochs=5):
    skf = StratifiedKFold()
    skf.get_n_splits(X, y)
    sumofsquares = np.zeros([13])
    sumofweights = np.zeros([13])
    for train_index, test_index in skf.split(X, y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)
        model = keras.models.Sequential()
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        for i, layer in enumerate(hidden_layers):
            if i == 0:
                model.add(keras.layers.Dense(units=layer, input_dim=13, activation=activation_function))
            else:
                model.add(keras.layers.Dense(units=layer, activation=activation_function))
        model.add(keras.layers.Dense(units=1, activation=activation_function))
        model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=keras.metrics.BinaryAccuracy()) #'sgd 'keras.metrics.BinaryAccuracy()
        model.summary()
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
        print("Test Results:")
        results = model.evaluate(X_test, y_test, batch_size=batch_size)
        for i, layer_size in enumerate(hidden_layers):
            for index in range(layer_size):
                print(f"hidden_layers[{i}] = index=[{index}] {model.weights[0].numpy()[:,index]}")
                sumofsquares += model.weights[0].numpy()[:,index]**2
                sumofweights += model.weights[0].numpy()[:,index]
                #print(f"model.weights len = {len(model.weights)}")
    print("absvalues=")
    print(absvalues)
    return sumofsquares,sumofweights